<a href="https://colab.research.google.com/github/Chan0226/Reinforcement-learning-game/blob/main/%EA%B0%95%ED%99%94%ED%95%99%EC%8A%B5%EA%B3%BC_%EA%B2%8C%EC%9E%841.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 다중손잡이 밴딧 문제 - [ 행동 - 보상] 사이클

In [ ]:
import numpy as np

In [ ]:
# 이문제는 확율을 모르면서 매 게임을 할때 얻는 이익을 보고 최대한 이익을 찾을수 있는 방향을...
# 이 알고리즘의 정보는 행동에 대한 보상(1 -1)
# 밴딩머신의 손잡이 승률
arms_profit = [0.4,0.12,0.52,0.6,0.25]
n_arms = len(arms_profit)

#손잡이를 당기는 횟수 ( 에피소드의 길이)  문제의 시작과 끝까지 도달하는데 시도하는 횟수 또는 합습횟수
n_trial = 10000

#handle : 손잡이 번호(게임기 번호)
def pull_banit(handle):
  q = np.random.random()
  if q < arms_profit[handle]:  
    return 1
  else:
    return -1  

# 랜덤정책을 모방하는 함수
def random_exploration():
  espisode=[]
  num = np.zeros(n_arms)    # 손잡이별로 당긴 횟수
  wins  = np.zeros(n_arms)  # 손잡이별로 이긴 횟수
  for i in range(n_trial):
    h = np.random.randint(0,n_arms) # 무작위로 손잡이를 선택
    reward = pull_banit(h)
    espisode.append([h,reward])
    num[h] += 1
    wins[h] +=1 if reward == 1 else 0 
  return espisode,(num,wins)

In [ ]:
e,r =  random_exploration()
# 손잡이별 승리확율
# 이긴횟수 / 총 시도 횟수    win /  num     r[1] / r[0]

# 아래 결과의 백터가 프로그램이 찾은 최적의 정책
print('손잡이별 승리 확율 : ', ['%.4f' % (r[1][i] / r[0][i]) for i in range(n_arms)] )

# 손잡이별 수익
# r[0][i] - r[1][i] for i in range(n_arms)
print('손잡이별 수익($) : ', ['%d' % (2*r[1][i] - r[0][i]) for i in range(n_arms)] )
print('순수익 :',sum(np.asarray(e)[:,1]))


손잡이별 승리 확율 :  ['0.3906', '0.1177', '0.5302', '0.5882', '0.2582']
손잡이별 수익($) :  ['-438', '-1552', '120', '360', '-940']
순수익 : -2450


In [ ]:
# 최초 설정한 확율과 arms_profit = [0.4,0.12,0.52,0.6,0.25] 와 결과가 비슷
# [0,0,0,1,0] --> 확율을 몰아준 최적의 정책

In [ ]:
# 입실론 탐욕 알고리즘
def epsilon_greedy(eps):
  episode = []
  num = np.zeros(n_arms) # 손잡이 당긴 횟수
  wins = np.zeros(n_arms) # 손잡이 별로 이긴 횟수
  for i in range(n_trial):
    r = np.random.random()  # 랜덤방식 -- 탐험형의 기본..
    if(r < eps or sum(wins) == 0 ):           # 탐험형 
      h = np.random.randint(0,n_arms)     # 임의의 핸들(게임기)
    else:                                     # 탐사형   높은 확율을 보이는 경우만 계속 시도하는 --> 보완... 일정비율을 추가해서 탐험형도 시도
      prob = np.asarray([wins[i] / num[i]  if num[i] > 0 else 0.0 for i in range(n_arms)])
      prob = prob / sum(prob)
      h = np.random.choice(range(n_arms),p=prob)
    reward = pull_banit(h)
    episode.append([h,reward])
    num[h] += 1
    wins[h] += 1 if reward == 1 else 0
  return episode, (num,wins)     
  



In [ ]:
e,r = epsilon_greedy(0.1)
print('손잡이별 승리 확율 : ', ['%.4f' % (r[1][i] / r[0][i]) for i in range(n_arms)] )
print('손잡이별 수익($) : ', ['%d' % (2*r[1][i] - r[0][i]) for i in range(n_arms)] )
print('순수익 :',sum(np.asarray(e)[:,1]))

손잡이별 승리 확율 :  ['0.3886', '0.1160', '0.5225', '0.5957', '0.2347']
손잡이별 수익($) :  ['-451', '-510', '129', '585', '-737']
순수익 : -984


In [ ]:
# 탐욕 알고리즘  - 보완예정
def greedy():
  episode = []
  num = np.zeros(n_arms) # 손잡이 당긴 횟수
  wins = np.zeros(n_arms) # 손잡이 별로 이긴 횟수
  for i in range(n_trial):
    r = np.random.random()  # 랜덤방식 -- 탐험형의 기본..
    if(sum(wins) == 0 ):           
      h = np.random.randint(0,n_arms)     # 임의의 핸들(게임기)
    else:                                     # 탐사형   높은 확율을 보이는 경우만 계속 시도하는 --> 보완... 일정비율을 추가해서 탐험형도 시도
      prob = np.asarray([wins[i] / num[i]  if num[i] > 0 else 0.0 for i in range(n_arms)])
      prob = prob / sum(prob)
      h = np.random.choice(range(n_arms),p=prob)
    reward = pull_banit(h)
    episode.append([h,reward])
    num[h] += 1
    wins[h] += 1 if reward == 1 else 0
  return episode, (num,wins)     

In [ ]:
e,r = greedy()
print('손잡이별 승리 확율 : ', ['%.4f' % (r[1][i] / r[0][i]) for i in range(n_arms)] )
print('손잡이별 수익($) : ', ['%d' % (2*r[1][i] - r[0][i]) for i in range(n_arms)] )
print('순수익 :',sum(np.asarray(e)[:,1]))

손잡이별 승리 확율 :  ['0.0000', '0.0000', '0.5293', '0.0000', 'nan']
손잡이별 수익($) :  ['-1', '-2', '586', '-1', '0']
순수익 : 582


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
